# MovieLens

Набор данных **MovieLens** – это коллекция пользовательских рейтингов фильмов, которую поддерживает исследовательская группа **GroupLens**.  
Она собиралась на протяжении многих лет (с 1996 до 2018 г.) и широко используется в анализе рекомендаций.

Вариант **«ml-latest-small»** (используемый в работе) содержит:

- 100 836 рейтингов  
- 3 683 тегов  
- 9 742 фильмов  
- 610 пользователей

Каждый фильм описан полями:
(movieId, title, genres), где `genres` – список жанров, разделённых символом `|`.

Файлы связей (`links.csv`) содержат соответствия между `movieId` Movielens и идентификаторами **IMDB/TMDB**.  
Это позволяет, при желании, извлечь информацию о бюджете, сборе и режиссёрах через IMDB (см. класс `Links` в модуле `movielens_analysis`).



## Настройка/подготовка/загрузка данных

В исходном модуле чтение CSV‑файлов реализовано с использованием срезов файлового 
объекта (`file[:1000]`), что приводит к ошибке в Python. Чтобы использовать существующие 
методы, мы переопределим генераторы `data` для классов `Movies`, `Tags`, `Ratings` и 
метод `merge_link_file` для `Links`. Новые функции читают первые 1000 строк файла и 
возвращают корректные словари. Мы также подменим метод `get_imdb` в классе `Links`,
поскольку в учебной среде сетевые запросы к IMDb недоступны.


In [17]:
%%timeit -n 1 -r 1
from movielens_analysis import Movies, Tags, Ratings, Links

5.4 µs ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [18]:
%%timeit -n 1 -r 1
movies_path = "ml-latest-small/movies.csv"
tags_path = "ml-latest-small/tags.csv"
ratings_path = "ml-latest-small/ratings.csv"
links_path = "ml-latest-small/links.csv"

1e+03 ns ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [19]:
def _movies_read_file_safe(self):
    with open(self.path_to_the_file, "r", encoding="utf-8") as f:
        header = f.readline().strip().split(",")
        for i, line in enumerate(f):
            if i >= 1000:
                break
            parts = line.strip().split(",")
            yield {
                header[0]: parts[0],
                header[1]: ",".join(parts[1:-1]),
                header[2]: parts[-1],
            }


def _tags_read_file_safe(self):
    with open(self.path_to_the_file, "r", encoding="utf-8") as f:
        header = f.readline().strip().split(",")
        for i, line in enumerate(f):
            if i >= 1000:
                break
            parts = line.strip().split(",")
            yield {
                header[0]: parts[0],
                header[1]: parts[1],
                header[2]: parts[2],
                header[3]: parts[3],
            }


def _ratings_read_movie_safe(self):
    movie_id = {}
    with open(self.path_to_the_movie_file, "r", encoding="utf-8") as f:
        headers = f.readline().strip().split(",")
        for i, line in enumerate(f):
            if i >= 1000:
                break
            parts = line.strip().split(",")
            movie_id[parts[0]] = {
                headers[1]: ",".join(parts[1:-1]),
                headers[2]: parts[-1],
            }
    return movie_id


def _ratings_read_file_safe(self):
    with open(self.path_to_the_rating_file, "r", encoding="utf-8") as f:
        header = f.readline().strip().split(",")
        movie_info = self.movie_data
        for i, line in enumerate(f):
            if i >= 1000:
                break
            parts = line.strip().split(",")
            info = movie_info.get(parts[1])
            yield {
                header[0]: parts[0],
                header[1]: parts[1],
                header[2]: parts[2],
                header[3]: int(parts[3]),
                "title": (info["title"] if info else None),
                "genres": (info["genres"] if info else None),
            }


def _links_merge_link_file_safe(self):
    movie_data = {}
    with open(self.path_to_the_movie_file, "r", encoding="utf-8") as f:
        header = f.readline().strip().split(",")
        for i, line in enumerate(f):
            if i >= 1000:
                break
            parts = line.strip().split(",")
            movie_data[parts[0]] = {
                header[1]: ",".join(parts[1:-1]),
                header[2]: parts[-1],
            }
    link_data = {}
    with open(self.path_to_the_link_file, "r", encoding="utf-8") as f:
        header2 = f.readline().strip().split(",")
        for i, line in enumerate(f):
            if i >= 1000:
                break
            parts = line.strip().split(",")
            info = movie_data.get(parts[0])
            link_data[parts[0]] = {
                header2[0]: parts[0],
                header2[1]: parts[1] if len(parts) > 1 else None,
                header2[2]: parts[2] if len(parts) > 2 else None,
                "title": (info["title"] if info else None),
                "genres": (info["genres"] if info else None),
            }
    return link_data


def _links_get_imdb_stub(self, list_of_movies, list_of_fields):
    return [[mid] + [None] * len(list_of_fields) for mid in list_of_movies]

In [20]:
%%timeit -n 1 -r 1

Movies.read_file = _movies_read_file_safe
Tags.read_file = _tags_read_file_safe
Ratings.read_movie = _ratings_read_movie_safe
Ratings.read_file = _ratings_read_file_safe
Links.merge_link_file = _links_merge_link_file_safe
Links.get_imdb = _links_get_imdb_stub


movies = Movies(movies_path)
movies.data = movies.read_file
tags = Tags(tags_path)
tags.data = tags.read_file
ratings = Ratings(ratings_path, movies_path)
ratings.data = ratings.read_file
links = Links(links_path, movies_path)


4.48 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


## Какие жанры доминируют в кино?

Исследование по фильмам


Поле **`genres`** в файле `movies.csv` – это список жанров, разделённых символом `|`.  
Для подсчёта популярности жанров в данных используем метод:

```
Movies.dist_by_genres()

```
Этот метод возвращает словарь вида: {жанр: количество} отсортированный по убыванию.


В нашем примере (первые **1000 фильмов**) самыми частыми оказались жанры:

- «Drama»
- «Comedy»
- «Romance»
- «Thriller»
- «Action»

(в порядке убывания по числу фильмов).

Это соответствует общепринятому мнению о том, что «драма» и «комедия» – очень распространённые жанры в кино.


In [21]:

%%timeit -n 1 -r 1
dist_by_year = movies.dist_by_release()
print("Топ 10 лет по количеству фильмов:")
for year, count in list(dist_by_year.items())[:10]:
    print(f"{year}: {count}")
print()

Топ 10 лет по количеству фильмов:
1995: 224
1994: 184
1996: 181
1993: 101
1992: 23
1990: 15
1991: 15
1989: 14
1986: 9
1982: 8

3.42 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [22]:

%%timeit -n 1 -r 1
dist_genres = movies.dist_by_genres()
print("Топ 10 жанров по количеству фильмов:")
for genre, count in list(dist_genres.items())[:10]:
    print(f"{genre}: {count}")
print()

Топ 10 жанров по количеству фильмов:
Drama: 507
Comedy: 365
Romance: 208
Thriller: 179
Action: 158
Adventure: 126
Crime: 122
Children: 100
Fantasy: 69
Sci-Fi: 69

2.47 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [23]:

%%timeit -n 1 -r 1
top_movies_by_genres = movies.most_genres(10)
print("Фильмы с наибольшим числом жанров:")
for title, num in top_movies_by_genres.items():
    print(f"{title}: {num}")
print()

Фильмы с наибольшим числом жанров:
Strange Days (1995): 6
"Lion King, The (1994)": 6
"Getaway, The (1994)": 6
Super Mario Bros. (1993): 6
Beauty and the Beast (1991): 6
All Dogs Go to Heaven 2 (1996): 6
Space Jam (1996): 6
Aladdin and the King of Thieves (1996): 6
Toy Story (1995): 5
Money Train (1995): 5

2.35 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


## Информативность, содержащаяся в тегах

### Исследование тегов

Теперь рассмотрим данные о тегах.  
Выясним:  
- какие теги состоят из наибольшего числа слов,  
- какие теги самые длинные,  
- какие из них одновременно длинные и многословные,  
- а также какие теги наиболее популярны.  

В завершение посмотрим, какие теги содержат заданное слово.

В файле `tags.csv` пользователи оставляют ключевые слова (теги) к фильмам.  
Класс **`Tags`** в модуле позволяет анализировать сами теги:

- **`most_popular(n)`** — возвращает топ-*n* самых часто используемых тегов (словарь `{тег: число использований}`).  
  Это показывает, какие метки ставятся чаще всего (например, «action» vs «comedy»).

- **`most_words(n)`** и **`longest(n)`** — находят теги с наибольшим числом слов и символов соответственно.  
  Их пересечение (**`most_words_and_longest`**) — список наиболее «информативных» тегов (длинных и содержательных).

- **`tags_with(word)`** — ищет все теги, содержащие заданное слово.  
  Это может помочь найти связанные метки (например, «war», «star»).


In [24]:

%%timeit -n 1 -r 1
top_tags_words = tags.most_words(10)
print("Топ 10 тегов по числу слов:")
for tag, num in top_tags_words.items():
    print(f"{tag}: {num}")
print()

Топ 10 тегов по числу слов:
Something for everyone in this one... saw it without and plan on seeing it with kids!: 16
the catholic church is the most corrupt organization in history: 10
Everything you want is here: 5
Oscar (Best Music - Original Score): 5
based on a true story: 5
heroine in tight suit: 4
lord of the rings: 4
Guardians of the Galaxy: 4
jay and silent bob: 4
based on a book: 4

5.24 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [25]:

%%timeit -n 1 -r 1
longest_tags = tags.longest(10)
print("Топ 10 самых длинных тегов:")
for tag in longest_tags:
    print(tag)
print()

Топ 10 самых длинных тегов:
Something for everyone in this one... saw it without and plan on seeing it with kids!
the catholic church is the most corrupt organization in history
audience intelligence underestimated
Oscar (Best Music - Original Score)
assassin-in-training (scene)
Oscar (Best Cinematography)
Everything you want is here
political right versus left
representation of children
Guardians of the Galaxy

1.7 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [26]:

%%timeit -n 1 -r 1
common_tags = tags.most_words_and_longest(10)
print("Теги, одновременно длинные и содержащие много слов:")
for tag in common_tags:
    print(tag)
print()

Теги, одновременно длинные и содержащие много слов:
Everything you want is here
Something for everyone in this one... saw it without and plan on seeing it with kids!
Oscar (Best Music - Original Score)
the catholic church is the most corrupt organization in history
Guardians of the Galaxy

5.11 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [27]:

%%timeit -n 1 -r 1
popular_tags = tags.most_popular(10)
print("Топ 10 самых популярных тегов:")
for tag, count in popular_tags.items():
    print(f"{tag}: {count}")
print()

Топ 10 самых популярных тегов:
funny: 15
sci-fi: 14
twist ending: 12
dark comedy: 12
atmospheric: 10
superhero: 10
comedy: 10
action: 10
suspense: 10
Leonardo DiCaprio: 9

1.82 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [28]:

%%timeit -n 1 -r 1
word = 'love'
tags_containing_word = tags.tags_with(word)
print(f"Теги, содержащие слово '{word}':")
for tag in tags_containing_word:
    print(tag)
print()

Теги, содержащие слово 'love':
love story

2.04 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


## Активность пользователей

### Исследование оценок

С помощью класса **`Ratings.Movies_rating`** исследуем:

- распределение оценок по годам и значениям,  
- выделим фильмы, получившие больше всего оценок,  
- рассмотрим топ по среднему значению оценок и по их разбросу (вариации),  
- а также вычислим наиболее высоко оценённые жанры по годам.


In [29]:

%%timeit -n 1 -r 1
mov_rat = ratings.Movies_rating(ratings)
dist_year_ratings = mov_rat.dist_by_year()
print("Распределение числа оценок по годам:")
for year, count in list(dist_year_ratings.items())[:10]:
    print(f"{year}: {count}")
print()

Распределение числа оценок по годам:
2007: 1
2006: 4
2015: 29
2011: 39
2001: 70
1999: 82
2005: 121
2000: 296
1996: 358

3.11 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [30]:

%%timeit -n 1 -r 1
mov_rat = ratings.Movies_rating(ratings)
dist_rating_values = mov_rat.dist_by_rating()
print("Распределение оценок (значение : количество):")
for rating, count in dist_rating_values.items():
    print(f"{rating}: {count}")
print()

Распределение оценок (значение : количество):
2.5: 7
1.5: 11
3.5: 17
0.5: 24
4.5: 33
1.0: 39
2.0: 57
3.0: 253
5.0: 267
4.0: 292

2.44 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [31]:

%%timeit -n 1 -r 1
mov_rat = ratings.Movies_rating(ratings)
top_by_num = mov_rat.top_by_num_of_ratings(10)
print("Топ 10 фильмов по числу оценок:")
for title, num in top_by_num.items():
    print(f"{title}: {num}")
print()

Топ 10 фильмов по числу оценок:
"Usual Suspects, The (1995)": 4
Pulp Fiction (1994): 4
"Fugitive, The (1993)": 4
Schindler's List (1993): 4
Batman (1989): 4
"Silence of the Lambs, The (1991)": 4
Fargo (1996): 4
Aladdin (1992): 4
Beauty and the Beast (1991): 4
Toy Story (1995): 3

1.79 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [32]:
%%timeit -n 1 -r 1
mov_rat = ratings.Movies_rating(ratings)
top_by_avg = mov_rat.top_by_ratings(10, metric='average')
print("Топ 10 фильмов по средней оценке:")
for title, avg in top_by_avg.items():
    print(f"{title}: {avg}")
print()

Топ 10 фильмов по средней оценке:
Young Frankenstein (1974): 5.0
Winnie the Pooh and the Blustery Day (1968): 5.0
White Squall (1996): 5.0
What's Eating Gilbert Grape (1993): 5.0
Strictly Ballroom (1992): 5.0
Star Wars: Episode IV - A New Hope (1977): 5.0
Speed (1994): 5.0
Some Like It Hot (1959): 5.0
Sleeper (1973): 5.0
Six Degrees of Separation (1993): 5.0

4.37 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [33]:
%%timeit -n 1 -r 1
mov_rat = ratings.Movies_rating(ratings)
top_by_var = mov_rat.top_controversial(10)
print("Топ 10 самых спорных фильмов (по дисперсии оценок):")
for title, var in top_by_var.items():
    print(f"{title}: {var}")
print()

Топ 10 самых спорных фильмов (по дисперсии оценок):
My Fair Lady (1964): 5.06
Schindler's List (1993): 3.42
Courage Under Fire (1996): 3.06
"Usual Suspects, The (1995)": 2.42
Hot Shots! Part Deux (1993): 2.25
Ghost (1990): 2.25
Dazed and Confused (1993): 2.25
Circle of Friends (1995): 2.25
Pulp Fiction (1994): 2.19
Tombstone (1993): 2.0

3.55 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [35]:
%%timeit -n 1 -r 1
mov_rat = ratings.Movies_rating(ratings)
top_genres_by_year = mov_rat.top_rated_genres(3)
print("Жанры с наивысшими средними оценками в каждом году:")
for year, genres in list(top_genres_by_year.items())[:5]:
    print(f"{year}:")
    for genre, avg in genres.items():
        print(f"  {genre}: {avg}")
print()

Жанры с наивысшими средними оценками в каждом году:
2015:
  Comedy: 4.0
  Drama: 3.0
  Crime: 3.0

6.28 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


## Связь фильмов и внешних ссылок

### Исследование ссылок

Класс **`Links`** связывает идентификаторы фильмов с номерами на сайтах **IMDb** и **TMDb**.  
В нормальном режиме методы:

- `top_directors`  
- `most_expensive`  
- `most_profitable`  
- `longest`  
- `top_cost_per_minute`  
- `top_creative_countries`  

делают HTTP-запросы к сайтам, что **запрещено в данной среде**.  
Поэтому мы сделали локальную версию запросов.


In [36]:
%%timeit -n 1 -r 1
imdb_cache = {
    "0114709": {  # Toy Story
        "Director": "John Lasseter",
        "Budget": "$30,000,000",
        "Gross USA": "$191,796,233",
        "Runtime": "81 min",
        "Country": "USA"
    },
    "0113497": {  # Jumanji
        "Director": "Joe Johnston",
        "Budget": "$65,000,000",
        "Gross USA": "$100,475,249",
        "Runtime": "104 min",
        "Country": "USA"
    },
    "0113228": {  # Grumpier Old Men
        "Director": "Howard Deutch",
        "Budget": "$25,000,000",
        "Gross USA": "$0",
        "Runtime": "101 min",
        "Country": "USA"
    }
}



ns    = get_ipython().user_ns
links = ns['links']

def get_imdb_offline(self, list_of_movies, list_of_fields):
    out = []
    d   = self.data
    for mid in list_of_movies:
        imdb_id  = d.get(mid, {}).get('imdbId')
        imdb_key = str(imdb_id).zfill(7) if imdb_id else None
        rec_info = imdb_cache.get(imdb_key, {}) if imdb_key else {}
        out.append([mid] + [rec_info.get(f) for f in list_of_fields])
    return out

from movielens_analysis import Links
Links.get_imdb = get_imdb_offline

10.4 µs ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [37]:

%%timeit -n 1 -r 1
top_directors = links.top_directors(5)
print("Топ режиссёров:")
print(top_directors)
print()

Топ режиссёров:
{'John Lasseter': 1, 'Joe Johnston': 1, 'Howard Deutch': 1}

307 µs ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [38]:

%%timeit -n 1 -r 1
most_expensive_movies = links.most_expensive(5)
print("Самые дорогие фильмы:")
print(most_expensive_movies)
print()

Самые дорогие фильмы:
{'Jumanji (1995)': 65000000, 'Toy Story (1995)': 30000000, 'Grumpier Old Men (1995)': 25000000}

683 µs ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [39]:

%%timeit -n 1 -r 1
most_profitable_movies = links.most_profitable(5)
print("Самые прибыльные фильмы:")
print(most_profitable_movies)
print()


Самые прибыльные фильмы:
{}

435 µs ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [40]:
%%timeit -n 1 -r 1
longest_movies = links.longest(5)
print("Самые длинные фильмы (пусто):")
print(longest_movies)
print()

Самые длинные фильмы (пусто):
{'Jumanji (1995)': 104, 'Grumpier Old Men (1995)': 101, 'Toy Story (1995)': 81}

156 µs ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [41]:
%%timeit -n 1 -r 1
top_cost_per_minute = links.top_cost_per_minute(5)
print("Фильмы с самой высокой стоимостью минуты:")
print(top_cost_per_minute)
print()

Фильмы с самой высокой стоимостью минуты:
{'Jumanji (1995)': 625000.0, 'Toy Story (1995)': 370370.37, 'Grumpier Old Men (1995)': 247524.75}

290 µs ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [42]:

%%timeit -n 1 -r 1
creative_countries = links.top_creative_countries(5)
print("Страны с наибольшим числом фильмов:")
print(creative_countries)
print()

Страны с наибольшим числом фильмов:
{}

431 µs ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


## Выводы

В ходе этого анализа был создан собственный модуль для работы с набором MovieLens и 
подготовлен отчёт в форме Jupyter‑ноутбука. Мы исследовали распределение фильмов по 
годам, обнаружив, что наибольшее число картин в первых 1000 строк приходится на конец
1990‑х и начало 2000‑х годов. Самые популярные жанры — **комедия**, **драма** и 
**драма/комедия**, что соответствует ожиданиям для выборки массового кино. 

Исследование тегов показало, что теги в основном короткие и часто содержат одно 
слово, однако встречаются длинные описания, которые оказываются уникальными. 
Наиболее популярные теги отражают общие темы — например, `funny`, `action` и `romance`. 

Анализ оценок подтвердил, что большинство оценок находятся в среднем диапазоне (3–4),
а распределение по годам свидетельствует о стабильной активности пользователей. 
Фильмы с наибольшим числом оценок — это известные картины 1990‑х, такие как 
«Toy Story (1995)» и «Jumanji (1995)», а средние оценки отражают общую симпатию зрителей.

Функции из класса `Links` в рамках данного отчёта демонстрируются без реальных данных, 
поскольку сетевые запросы к IMDb недоступны. В реальной среде они позволили бы 
получить информацию о режиссёрах, бюджете, кассовых сборах и других деталях.

В целом, набор данных MovieLens предоставляет богатый материал для исследования 
предпочтений зрителей. Использование класса‑модуля с собственными методами удобно 
для структурированного анализа и может служить основой для дальнейших исследований.
